<a href="https://colab.research.google.com/github/yorkjong/vistock/blob/feature%2Fibd/notebooks/ibd_rs_rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stock Analysis and Ranking with IBD RS Rating, inspired by the Investor's Business Daily (IBD) methodology.

### Install and Setup (免費版Colab會固定時間清掉安裝的東西，所以重安裝是新連線後最先要做的事)

#### Install Packages

In [1]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/ibd"
%pip install requests-cache

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/ibd) to /tmp/pip-req-build-lnhhqlkd
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-lnhhqlkd
  Running command git checkout -b feature/ibd --track origin/feature/ibd
  Switched to a new branch 'feature/ibd'
  Branch 'feature/ibd' set up to track remote branch 'feature/ibd' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit 3464467af44286e8657f1a0a42257831939abe82
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.6 MB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.4.0-py3-none-any.whl size=42392 sha256=6668a2571803d10c5559b08a7e9d249fad2b7a86e9311441973ee18714878ada
  Stored in directory: /tmp/pip-ephem-wheel-cache-1tdttfbp/wheels/07/97/7f/a72c7fbc975be0326ac45fec1ca0c872e2d6e2bbea96caf706
Successfully built vistock
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4

#### Setup and Configuration

In [2]:
# @title Enable DataFrame Formatter
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [3]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [4]:
# @title Initialize Widgets
import ipywidgets as widgets
output = widgets.Output()

In [5]:
# @title GitHub
import base64
import requests
import pandas as pd
from io import StringIO


class GitHub:
    def __init__(self, repo_owner, repo_name, token, dir='', branch='main'):
        base = 'https://api.github.com/repos'
        dir = dir.strip('/')
        if dir:
            self.base_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.base_url = f'{base}/{repo_owner}/{repo_name}/contents'
        self.branch = branch
        self.token = token

    def _request(self, method, url, headers=None, params=None, json=None):
        response = requests.request(method, url, headers=headers,
                                    params=params, json=json)
        if response.status_code in [200, 201]:
            return response.json()
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return None

    def file_exists(self, file_path):
        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }
        response = requests.get(url, headers=headers,
                                params={'ref': self.branch})
        if response.status_code == 200:
            return True
        elif response.status_code == 404:
            return False
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return None

    def list_filenames(self, dir_path=''):
        url = f'{self.base_url}/{dir_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }
        response = requests.get(url, headers=headers,
                                params={'ref': self.branch})
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return []

    def download_file(self, file_path):
        if not self.file_exists(file_path):
            print(f"File '{file_path}' does not exist. Cannot download.")
            return None

        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }

        file_info = self._request('GET', url, headers=headers)
        if file_info:
            download_url = file_info['download_url']
            response = requests.get(download_url)
            if response.status_code == 200:
                return StringIO(response.text)
            else:
                print(f"Failed to download file: "
                      f"{response.status_code} - {response.text}")
                return None
        return None

    def download_csv(self, file_path):
        file_content = self.download_file(file_path)
        if file_content:
            return pd.read_csv(file_content)
        else:
            return pd.DataFrame()

    def upload_file(self, file_path, content):
        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json'
        }

        encoded_content = base64.b64encode(content.encode()).decode()
        payload = {
            'message': 'Uploading file',
            'content': encoded_content,
            'branch': self.branch
        }
        self._request('PUT', url, headers=headers, json=payload)

    def upload_df_as_csv(self, file_path, df):
        """Upload a DataFrame to a CSV file."""
        if not file_path.endswith('.csv'):
            file_path += '.csv'
        csv_content = df.to_csv(index=False)
        self.upload_file(file_path, csv_content)

    def remove_file(self, file_path):
        if not self.file_exists(file_path):
            print(f"File '{file_path}' does not exist. Skipping deletion.")
            return

        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json'
        }

        # Fetch the file info to get the SHA needed for deletion
        file_info = self._request('GET', url, headers=headers)
        if file_info:
            sha = file_info['sha']
            payload = {
                'message': 'Deleting file',
                'sha': sha,
                'branch': self.branch
            }
            self._request('DELETE', url, headers=headers, json=payload)

#-------------------------------------------------------------------------------

from google.colab import userdata

github = GitHub(
    repo_owner='YorkJong',
    repo_name='stock-reports',
    token=userdata.get('GithubToken.stock-reports'),
    dir='ibd',
)

In [6]:
# @title Update and Filter DataFrame

def update_tickers_with_names(df, ticker_column, name_separator=','):
    """
    Update ticker codes in a DataFrame with their corresponding stock names.

    Parameters:
    - df: The DataFrame containing ticker codes.
    - ticker_column: The name of the column containing ticker codes.
    - name_separator: Separator used to join names (default is comma for multiple tickers).

    This function updates the specified column with the stock names instead of ticker codes.
    """
    # Iterate over the specified column in the DataFrame
    for index, row in df.iterrows():
        tickers = row[ticker_column].split(name_separator)  # Split the tickers string into a list
        stock_names = [tw.stock_name(ticker) for ticker in tickers]  # Get stock names for each ticker
        # Update the stock names back to the DataFrame
        df.at[index, ticker_column] = name_separator.join(stock_names)  # Join the names back into a string


def remove_ticker_suffix(df, ticker_column):
    """
    Remove the '.TW' or '.TWO' suffix from ticker codes in a DataFrame.

    Parameters:
    - df: The DataFrame containing ticker codes.
    - ticker_column: The name of the column containing ticker codes.
    """
    # Apply string replacement for each ticker in the specified column
    df[ticker_column] = df[ticker_column].str.replace('.TWO', '', regex=False)
    df[ticker_column] = df[ticker_column].str.replace('.TW', '', regex=False)


def filter_increasing_relative_strength(df):
    """
    Filter stocks with increasing Relative Strength over different time periods.

    This function filters the DataFrame to include only those stocks where:
    - Relative Strength is above 100.
    - Relative Strength has increased over the past 1 month, 3 months, and 6 months.
    Optionally, you can add a condition to check if Percentile is above 90.
    """
    return df[
        (df["Relative Strength"] > 100)
        & (df["Relative Strength"] > df["1 Month Ago"])
        & (df["1 Month Ago"] > df["3 Months Ago"])
        & (df["3 Months Ago"] > df["6 Months Ago"])
        # & (df["Percentile"] > 90)  # Uncomment to include Percentile filter
    ]


In [7]:
# @title Rank Function

import os
from datetime import datetime

from vistock import ibd
from vistock import tw
from vistock.stock_indices import get_tickers

def rank(code, period='2y', tickers_getter=get_tickers,
         ref_ticker='^GSPC', out_dir='out'):
    tickers = tickers_getter(code)

    output.clear_output()
    with output:
        rank_stock, rank_indust = ibd.rankings(tickers, period=period,
                                               ref_ticker=ref_ticker)
    if rank_stock.empty or rank_indust.empty:
        print("Not enough data to generate rankings.")
        return

    # Update the stock names back to the DataFrame
    #update_tickers_with_names(rank_stock, 'Ticker')
    #update_tickers_with_names(rank_indust, 'Tickers')

    # Remove the '.TW' or '.TWO' suffix
    remove_ticker_suffix(rank_stock, 'Ticker')
    remove_ticker_suffix(rank_indust, 'Tickers')

    # Save to CSV
    print("\n\n***")
    os.makedirs(out_dir, exist_ok=True)
    today = datetime.now().strftime('%Y%m%d')
    for df, kind in zip([rank_stock, rank_indust],
                           ['stocks', 'industries']):
        filename = f'{code}_{kind}_{period}_{today}.csv'
        github.upload_df_as_csv(filename, df)
        df.to_csv(os.path.join(out_dir, filename), index=False)
        print(f'Your "{filename}" is in the "{out_dir}" folder.')
    print("***\n")

    return rank_stock, rank_indust

### Glossary of Terms

source (The source of stocks to analyze):
- This could include stocks traded on exchanges or components of a specific index.
- Common abbreviation(s) for the exchange or market sector.  
  - For Taiwan Markets, possible values include:
    - `TWSE`: Taiwan Stock Exchange (台灣上市股票交易所）
    - `TPEX`: Taipei Exchange （上櫃交易所）
    - `ESB`: Emerging Stock Board （興櫃交易所）
  - Can also be combined with '+' (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`)
  - For America Markets, possible values include:
    - `SPX`: S&P 500 (標普五百指數)
    - `DJIA`: Dow Jones Industrial Average (道瓊指數)
    - `NDX`: NASDAQ-100 (納斯達克一百指數)
    - `SOX`: PHLX Semiconductor Index （費半指數）
  - Multiple indices can be combined using '+' (e.g., `SPX+DJIA+NDX+SOX`)

period (Historical Data Time Range)：
- The time range for which to fetch historical data.
- `2y` means 2 years
- `6mo` means 6 monthes

RS (Relative Strength)
- Relative Strength (RS) is a metric used to evaluate the performance of a stock relative to a benchmark index.
  - A higher RS rating indicates that the stock has outperformed the index, while a lower RS rating suggests underperformance.
- The IBD RS calculates the performance of the last year, with the most recent quarter weighted double.

min_percentile (最小百分位)
- The minimum percentile for a stock to be included in the rankings.

### Error Messages

In [8]:
display(output)

Output()

### RS Rating and Ranking

In [19]:
source = "All Indices" #@param ["S&P 500", "Dow Jones Industrial Average", "NASDAQ 100", "PHLX Semiconductor", "All Indices"]
period = "2y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    'S&P 500': 'SPX',
    'Dow Jones Industrial Average': 'DJIA',
    'NASDAQ 100': 'NDX',
    'PHLX Semiconductor': 'SOX',
    'All Indices': 'SPX+DJIA+NDX+SOX',
}

rank_stock, rank_indust = rank(code_from_name[source], period)
display(rank_stock)
display(rank_indust)



***
Your "SPX+DJIA+NDX+SOX_stocks_2y_20240813.csv" is in the "out" folder.
Your "SPX+DJIA+NDX+SOX_industries_2y_20240813.csv" is in the "out" folder.
***



,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
390,NVDA,Technology,Semiconductors,160.21,179.14,171.38,165.05,99,99,99,99,1
1,VST,Utilities,Utilities - Independent Power Producers,151.11,180.09,239.74,142.85,99,99,99,98,2
44,IRM,Real Estate,REIT - Specialty,146.08,127.62,116.55,105.31,99,96,89,72,3
206,GDDY,Technology,Software - Infrastructure,141.25,128.93,133.70,119.59,99,97,97,92,4
71,FICO,Technology,Software - Application,139.72,131.79,121.89,127.72,99,98,94,96,5
...,...,...,...,...,...,...,...,...,...,...,...,...
4,MDB,Technology,Software—Infrastructure,55.25,58.55,84.21,120.52,0,0,14,93,517
323,ALB,Basic Materials,Specialty Chemicals,52.20,56.92,79.55,61.72,0,0,7,0,518
376,INTC,Technology,Semiconductors,51.44,77.28,75.40,110.85,0,7,4,84,519
416,WBA,Healthcare,Pharmaceutical Retailers,46.27,46.84,66.75,74.63,0,0,0,3,520


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
1,Utilities - Independent Power Producers,Utilities,143.12,154.28,204.87,130.29,"VST,NRG",98,99,99,97,1
89,Medical Care Facilities,Healthcare,120.00,101.42,107.32,106.20,"UHS,DVA,HCA",97,83,92,80,2
65,Oil & Gas Midstream,Energy,116.77,106.97,112.07,98.96,"TRGP,OKE,KMI,WMB",96,94,96,53,3
76,REIT - Healthcare Facilities,Real Estate,115.46,100.02,99.38,89.10,"VTR,WELL,DOC",95,78,61,23,4
10,Tobacco,Consumer Defensive,113.07,99.25,99.42,85.50,"PM,MO",94,73,62,10,5
...,...,...,...,...,...,...,...,...,...,...,...,...
78,Steel,Basic Materials,82.02,87.36,99.08,100.67,"STLD,NUE",4,23,59,57,88
16,Auto Parts,Consumer Cyclical,77.80,77.41,86.64,77.76,"GPC,BWA,LKQ,APTV",3,1,6,1,89
64,Airlines,Industrials,75.58,84.39,99.46,94.57,"LUV,DAL,UAL,AAL",2,9,63,38,90
29,Resorts & Casinos,Consumer Cyclical,73.43,77.97,78.55,86.66,"MGM,CZR,LVS,WYNN",1,3,0,14,91


In [10]:
# @title Top Percentile Stocks
min_percentile = 5 # @param {"type":"slider","min":1,"max":99,"step":1}
top_stocks = rank_stock[rank_stock[ibd.TITLE_PERCENTILE] >= min_percentile]
num_rows, _ = top_stocks.shape
print(f'\nnumber of filtered tickers: {num_rows}')
top_stock_list = list(top_stocks["Ticker"])
print(top_stock_list)


number of filtered tickers: 472
['NVDA', 'VST', 'IRM', 'GDDY', 'FICO', 'HWM', 'NRG', 'TRGP', 'MMM', 'GE', 'AXON', 'UHS', 'KKR', 'NTAP', 'ANET', 'MHK', 'AVGO', 'ISRG', 'MPWR', 'LLY', 'VTR', 'RCL', 'TYL', 'META', 'K', 'COST', 'WELL', 'NEM', 'CEG', 'GRMN', 'PGR', 'EFX', 'CFG', 'GS', 'CTAS', 'DVA', 'TT', 'FSLR', 'CBRE', 'TMUS', 'MSI', 'BRO', 'HCA', 'RTX', 'AFL', 'IP', 'DECK', 'DHI', 'KLAC', 'PEG', 'PM', 'GLW', 'HIG', 'KEY', 'LMT', 'EBAY', 'REGN', 'T', 'FITB', 'PHM', 'NVR', 'ICE', 'CTLT', 'JNPR', 'BK', 'SO', 'MO', 'OKE', 'MCO', 'ABBV', 'EXR', 'LDOS', 'BSX', 'AMT', 'SYF', 'FANG', 'KMI', 'CPT', 'FIS', 'NFLX', 'IFF', 'FOX', 'TFC', 'APH', 'NDAQ', 'AXP', 'SMCI', 'EIX', 'JPM', 'ZBRA', 'CL', 'WMB', 'DUK', 'PNC', 'FOXA', 'WMT', 'NEE', 'URI', 'CHRW', 'IBM', 'PANW', 'ESS', 'NI', 'ALL', 'DOC', 'UBER', 'PWR', 'STX', 'FTNT', 'AEP', 'PNW', 'AAPL', 'LEN', 'TJX', 'NOW', 'MTB', 'RSG', 'MAA', 'WRB', 'CBOE', 'SPG', 'GNRC', 'PH', 'PKG', 'DFS', 'SPGI', 'PSA', 'GM', 'SHW', 'TDG', 'AJG', 'VRTX', 'ECL', 'RMD', 'N

In [11]:
# @title Filtered Stocks with Increasing RS > 100
filtered_rank_stock = filter_increasing_relative_strength(rank_stock)
display(filtered_rank_stock)

,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
41,IRM,Real Estate,REIT - Specialty,146.08,127.62,116.55,105.31,99,96,89,73,3
119,TRGP,Energy,Oil & Gas Midstream,133.87,123.28,121.62,108.02,98,95,93,78,8
194,VTR,Real Estate,REIT - Healthcare Facilities,119.60,101.32,97.97,86.95,95,71,52,21,21
413,TYL,Technology,Software - Application,119.34,109.05,102.68,97.67,95,87,66,54,23
251,WELL,Real Estate,REIT - Healthcare Facilities,118.23,103.46,102.70,101.59,94,75,66,63,27
183,NEM,Basic Materials,Gold,118.05,103.90,103.11,74.87,94,76,67,4,28
348,FSLR,Technology,Solar,115.87,109.14,98.16,76.74,92,87,53,4,38
271,MSI,Technology,Communication Equipment,115.15,106.39,104.48,96.53,91,81,71,51,41
179,AFL,Financial Services,Insurance - Life,114.44,101.19,100.22,94.09,90,71,60,44,45
235,IP,Consumer Cyclical,Packaging & Containers,114.44,110.51,106.47,91.56,90,89,75,37,46


### RS Rating and Ranking for Taiwan Stocks

In [12]:
from vistock import tw

source = "上市+上櫃" #@param ["上市", "上櫃", "上市+上櫃", "興櫃", "全部"]
period = "2y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    '上市': 'TWSE',
    '上櫃': 'TPEX',
    '上市+上櫃': 'TWSE+TPEX',
    '興櫃': 'ESB',
    '全部': 'TWSE+TPEX+ESB'
}

tw_stocks, tw_industries = rank(code_from_name[source], period,
     tickers_getter=tw.get_tickers, ref_ticker='^TWII')
display(tw_stocks)
display(tw_industries)



***
Your "TWSE+TPEX_stocks_2y_20240813.csv" is in the "out" folder.
Your "TWSE+TPEX_industries_2y_20240813.csv" is in the "out" folder.
***



,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
1115,6144,Communication Services,Entertainment,466.81,273.65,119.24,84.33,99.0,99.0,88,19,1
1674,1799,Healthcare,Drug Manufacturers—Specialty & Generic,404.50,386.05,220.02,187.96,99.0,99.0,99,97,2
1524,8374,Industrials,Industrial Distribution,384.72,266.61,85.74,83.32,99.0,99.0,45,16,3
798,2365,Technology,Computer Hardware,362.34,300.28,124.28,186.47,99.0,99.0,90,97,4
1502,4562,Industrials,Specialty Industrial Machinery,343.39,296.45,148.13,86.08,99.0,99.0,95,25,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1430,1312A,Basic Materials,Chemicals,NaN,68.34,73.76,82.45,NaN,7.0,13,14,2004
1459,2740,Consumer Cyclical,Restaurants,NaN,58.90,70.07,78.06,NaN,0.0,7,7,2005
1471,00682U,Unknown,Unknown,NaN,76.01,82.64,87.45,NaN,27.0,36,30,2006
1606,3226,Industrials,Electrical Equipment & Parts,NaN,77.30,77.78,93.68,NaN,32.0,21,50,2007


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
72,Industrial Distribution,Industrials,158.71,129.06,86.62,88.74,"8374,911608,3114,2373",99.0,97.0,41,27,1
56,Real Estate—Development,Real Estate,137.34,127.07,129.93,108.00,"2524,5508,3188,2537,5455,2718,1436,3489,4907,5...",98.0,96.0,99,86,2
65,Entertainment,Communication Services,132.54,104.77,92.79,96.67,"6144,4806,6596,8446,6184,8450,6464,6856,6625",97.0,90.0,63,64,3
8,Real Estate—Diversified,Real Estate,130.11,123.36,129.16,117.39,"1438,6219,9946,2520,2545,5512,2547",96.0,95.0,98,98,4
74,Utilities—Renewable,Utilities,121.79,130.67,101.12,114.32,"6869,6873,6806,8087",95.0,98.0,84,95,5
...,...,...,...,...,...,...,...,...,...,...,...,...
42,Chemicals,Basic Materials,NaN,80.21,82.67,88.94,"1312A,8438,1708,1709,8435,1735,1718,1714,1776,...",NaN,28.0,25,28,92
47,Medical Devices,Healthcare,NaN,79.25,81.72,92.81,"6762,4198,4129,4183,4735,1781,6598,1788,6649,4...",NaN,25.0,19,52,93
58,Specialty Business Services,Industrials,NaN,103.98,100.01,93.26,"8906,9929,8921,8401,6183,6721,8481,5607,6146",NaN,89.0,82,55,94
70,Restaurants,Consumer Cyclical,NaN,71.66,77.71,86.96,"3522,1268,2726,1259,2752,2729,2753,2755,2740,2...",NaN,4.0,9,15,95


In [13]:
# @title Top Percentile Taiwan Stocks
min_percentile = 95 # @param {"type":"slider","min":1,"max":99,"step":1}

top_stocks = tw_stocks[tw_stocks[ibd.TITLE_PERCENTILE] >= min_percentile]
num_rows, _ = top_stocks.shape
print(f'\nnumber of filtered tickers: {num_rows}')
top_stock_list = list(top_stocks["Ticker"])
top_stock_list = [tw.stock_name(ticker) for ticker in top_stock_list]
print(top_stock_list)


number of filtered tickers: 100
['得利影', '易威', '羅昇', '昆盈', '穎漢', '均華', '光聖', '皇昌', '海悅', '新復興', '康全電訊', '福大', '欣巴巴', '福裕', '所羅門', '晶彩科', '京城', '新門', '弘憶股', '翔耀', '慧友', '彬台', '慶騰', '太普高', '全譜', '擎亞', '均豪', '世紀', '福懋油', '東捷', '花王', '志聖', '天品', '順藥', '永信建', '弘塑', '鑫龍騰', '高鋒', '系微', '訊舟', '錦明', '鑫科', '安國', '訊聯基因', '京晨科', '三地開發', '聯上發', '昇益', '藥華藥', '聯鈞', '及成', '昇陽半導體', '和椿', '德晉', '天揚', '旺矽', '華城', '天方能源', '合騏', '藝舍-KY', '泰金-KY', '晶悅', '喬福', '華友聯', '鈊象', '晟銘電', '鏵友益', '盟立', '宏碩系統', '迎廣', '波力-KY', '惠特', '雲豹能源', '泰偉', '森寶', '富宇', '岱稜', '錸德', '大飲', '愛山林', '德律', '益登', '鈺邦', '天剛', '坤悅', '大城地產', '峰源-KY', '精湛', '信紘科', '勝昱', '華義', '萬潤', '富旺', '理銘', '一詮', '訊聯', '佳必琪', '佳能', '泓德能源-創', '宏太-KY']


In [14]:
# @title Filtered Taiwan Stocks with Increasing RS > 100

filtered_tw_stocks = filter_increasing_relative_strength(tw_stocks)
update_tickers_with_names(filtered_tw_stocks, 'Ticker')
display(filtered_tw_stocks)

filtered_tw_industries = filter_increasing_relative_strength(tw_industries)
update_tickers_with_names(filtered_tw_industries, 'Tickers')
display(filtered_tw_industries)

,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
1115,得利影,Communication Services,Entertainment,466.81,273.65,119.24,84.33,99.0,99.0,88,19,1
1674,易威,Healthcare,Drug Manufacturers—Specialty & Generic,404.50,386.05,220.02,187.96,99.0,99.0,99,97,2
1524,羅昇,Industrials,Industrial Distribution,384.72,266.61,85.74,83.32,99.0,99.0,45,16,3
1502,穎漢,Industrials,Specialty Industrial Machinery,343.39,296.45,148.13,86.08,99.0,99.0,95,25,5
815,均華,Technology,Semiconductors,317.98,291.08,240.56,196.52,99.0,99.0,99,98,6
...,...,...,...,...,...,...,...,...,...,...,...,...
1067,合邦,Technology,Semiconductors,103.26,90.88,83.87,69.88,78.0,63.0,40,1,419
768,新巨,Industrials,Electrical Equipment & Parts,102.93,101.35,101.08,95.06,78.0,76.0,73,54,424
50,磐亞,Basic Materials,Specialty Chemicals,101.60,98.54,93.30,92.56,77.0,74.0,62,47,446
224,業旺,Consumer Cyclical,Textile Manufacturing,101.05,97.90,93.17,81.31,77.0,73.0,61,12,456


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank


### Remove files in GitHub Repository

In [15]:
#for fn in [f for f in github.list_filenames() if f.endswith('.csv')]:
#    github.remove_file(fn)